<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/093_GAME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GAME

For this lecture, the **key focus** is the **GAME framework** itself — it’s a mental model for designing AI agents before you even write code.

---

## **What to Focus On**

### 1. **G — Goals & Instructions**

* **Goals** = *What* the agent is trying to achieve (the "win condition").
* **Instructions** = *How* the agent should go about achieving those goals.
* Grouped together because they shape the agent’s *decision-making foundation*.
* Your takeaway: The more concrete and constrained these are, the easier it is for the agent to operate without wandering off course.

---

### 2. **A — Actions**

* The **capabilities** available to the agent — the verbs it can use.
* Defined abstractly here, *not* as actual code yet.
* Example: `read_file()` is just a capability; how it works is up to the environment.

---

### 3. **M — Memory**

* What the agent remembers between iterations of the loop.
* Shapes what context is available for decision-making.
* Could be minimal (sliding window) or rich (long-term storage, vector search, file contents).

---

### 4. **E — Environment**

* Where and how the agent’s actions actually execute.
* Turns abstract actions into **real-world effects**.
* You can swap environments without changing the decision logic (e.g., local files → GitHub repo).

---

### **Key Insight:**

**Actions = Interface** (what you can do)
**Environment = Implementation** (how you actually do it)

This separation makes your agents **modular** and **adaptable** — you can change the environment without rewriting the decision-making logic.

---

### **Motivating Example: The Proactive Coder**

The Proactive Coder scenario shows how the GAME framework helps you:

* Scope goals and instructions tightly.
* Define only the actions needed for the job.
* Decide on a memory approach suited to the task.
* Match the environment implementation to the setting (local now, GitHub later).

---

### **Handbook-Worthy Takeaway**

> Always design with **GAME** before coding — it forces you to separate the *what* from the *how*, making agents easier to reason about, test, and adapt.



In [1]:
!pip install --quiet python-dotenv openai

In [2]:
import os, json
from openai import OpenAI
from dotenv import load_dotenv

# --- Load API key ---
load_dotenv('/content/API_KEYS.env')
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# --- GAME: Goals & Instructions ---
agent_config = {
    "goals": [
        "Summarize all .txt files in /content/files",
        "Write each summary to /content/summaries"
    ],
    "instructions": [
        "Be concise and clear.",
        "Skip unrelated text.",
        "Do not repeat file contents verbatim."
    ],
    "memory_window": 5
}

# --- Memory ---
messages = []

def append_chat_message(role, content):
    """Append a message to the conversation history."""
    messages.append({"role": role, "content": content})

def context_window():
    """Return the last N messages from chat history."""
    return messages[-agent_config["memory_window"]:]

# --- GAME: Environment ---
class LocalTxtEnvironment:
    base_dir = "/content/files"
    summaries_dir = "/content/summaries"

    @staticmethod
    def list_txt_files():
        return [f for f in os.listdir(LocalTxtEnvironment.base_dir) if f.endswith(".txt")]

    @staticmethod
    def read_txt(file_name):
        path = os.path.join(LocalTxtEnvironment.base_dir, file_name)
        if not os.path.exists(path):
            return {"error": f"File {file_name} not found"}
        with open(path, "r") as f:
            return {"content": f.read()}

    @staticmethod
    def write_summary_txt(file_name, content):
        os.makedirs(LocalTxtEnvironment.summaries_dir, exist_ok=True)
        path = os.path.join(LocalTxtEnvironment.summaries_dir, file_name)
        with open(path, "w") as f:
            f.write(content)
        return {"result": f"Summary saved to {path}"}

# --- Agent Decision Step ---
def agent_decision_cycle(user_input):
    """
    Send user input + context window to the model,
    store reply, and return assistant's decision text.
    """
    append_chat_message("user", user_input)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=context_window()
    )
    reply = response.choices[0].message.content
    append_chat_message("assistant", reply)
    return reply


This is actually one of the most important **GAME** concepts, and in my experience it’s the part people skip over when building agents, which leads to spaghetti code later.

---

## **What "Environment" Means in AI Agent Design**

In **GAME**, the *Environment* is **the real-world interface where your agent’s decisions become actions**.

Think of it like this:

* The *agent* is the brain: decides what to do (`list_txt_files`, `read_txt`, etc.).
* The *environment* is the body: actually does those actions in the real world.

---

### **Why We Make It a Separate Class**

Your environment is **where the agent "lives"**.
By separating it into its own class:

1. You make it **explicit** what the agent can actually do.
2. You can **swap environments** without touching your agent’s decision logic.

   * Today: `LocalTxtEnvironment` (local `/content/files` folder)
   * Tomorrow: `CloudTxtEnvironment` (reads from S3 or Google Drive)
3. You keep **implementation details** (e.g., file paths, error handling) away from the reasoning code.

---

### **Breaking Down Your Example**

```python
class LocalTxtEnvironment:
    base_dir = "/content/files"          # Where input text files live
    summaries_dir = "/content/summaries" # Where summaries get saved
```

These are **constants** that define *this* environment.
If you change the folder, you only change it here.

---

#### **1. list\_txt\_files**

```python
@staticmethod
def list_txt_files():
    return [f for f in os.listdir(LocalTxtEnvironment.base_dir) if f.endswith(".txt")]
```

This **action** is the environment’s way of saying:
"Here’s everything in my world that matches `.txt`."

---

#### **2. read\_txt**

```python
@staticmethod
def read_txt(file_name):
    path = os.path.join(LocalTxtEnvironment.base_dir, file_name)
    if not os.path.exists(path):
        return {"error": f"File {file_name} not found"}
    with open(path, "r") as f:
        return {"content": f.read()}
```

This lets the agent *perceive* something in the world — it reads a file and brings that info back into the agent’s "mind".

---

#### **3. write\_summary\_txt**

```python
@staticmethod
def write_summary_txt(file_name, content):
    os.makedirs(LocalTxtEnvironment.summaries_dir, exist_ok=True)
    path = os.path.join(LocalTxtEnvironment.summaries_dir, file_name)
    with open(path, "w") as f:
        f.write(content)
    return {"result": f"Summary saved to {path}"}
```

This is the agent *acting* on the world — changing it by creating a file.

---

### **How This Fits Into GAME**

* **Actions** = abstract capabilities (`list_txt_files`, `read_txt`, `write_summary_txt`).
* **Environment** = concrete implementation of those actions *in a specific setting*.
* If you wanted to move from **local files** to **cloud files**, the *agent logic* stays the same, only the *environment class* changes.

---

If you want, I can show you **side-by-side** how the same agent logic could run in **two completely different environments** just by swapping out the environment class — that’s when this concept really clicks.

Do you want me to do that?



Most agent tutorials on YouTube or GitHub mix **reasoning** and **doing** in one messy blob. That works for quick demos, but it becomes a nightmare the moment you try to:

* Run the same agent in a different environment (local → cloud → API).
* Debug why something failed (was it reasoning or execution?).
* Add new capabilities without breaking old ones.

---

### Why This Is a Big Deal

By **separating the mind from the body**:

* **Mind (Agent logic)** → Decides *what* to do.
* **Body (Environment)** → Knows *how* to do it in a specific setting.

That’s exactly how **traditional software design** works in clean architectures — and it’s how you make agents *maintainable, testable, and portable*.

---

### Benefits of This Separation

1. **Modularity** – Swap the environment without rewriting the agent.
2. **Testability** – You can mock the environment to test agent reasoning.
3. **Scalability** – Add new environments without breaking old ones.
4. **Clarity** – At a glance, you can see exactly what the agent can and can’t do.
5. **Focus** – The LLM "thinks" better when it’s not drowning in execution details.


Let’s step away from text files for a moment and imagine your agent is a **customer support bot** for an online store.

We’ll design an **Environment** that is *completely different* from the local file one, but still follows the same GAME principle.

---

## Example: **EcommerceSupportEnvironment**

Instead of local folders, the "world" for this agent is an **ecommerce platform API**.

```python
class EcommerceSupportEnvironment:
    """Environment for interacting with an ecommerce system."""

    # Simulated in-memory store database
    customers = {
        "C001": {"name": "Alice", "orders": ["O1001", "O1002"]},
        "C002": {"name": "Bob", "orders": []}
    }
    
    orders = {
        "O1001": {"status": "Shipped", "items": ["Laptop", "Mouse"]},
        "O1002": {"status": "Processing", "items": ["Headphones"]}
    }

    @staticmethod
    def list_customers():
        """Return all customer IDs."""
        return list(EcommerceSupportEnvironment.customers.keys())

    @staticmethod
    def get_customer_orders(customer_id):
        """Return all orders for a given customer."""
        customer = EcommerceSupportEnvironment.customers.get(customer_id)
        if not customer:
            return {"error": f"No customer found with ID {customer_id}"}
        return {"orders": customer["orders"]}

    @staticmethod
    def get_order_status(order_id):
        """Return the status and items for a specific order."""
        order = EcommerceSupportEnvironment.orders.get(order_id)
        if not order:
            return {"error": f"No order found with ID {order_id}"}
        return order
```

---

### How the Agent Would Use This Environment

* **Goal:** "Help customers track their orders."
* **Actions** (decided by the *mind*):

  * `list_customers`
  * `get_customer_orders`
  * `get_order_status`
* **Execution** (done by the *body*): The environment actually fetches the data.

---

### What’s Happening Here

* **Encapsulation:** All ecommerce logic lives *inside* the `EcommerceSupportEnvironment` — the agent doesn’t know if it’s using a fake database, a REST API, or a CSV file.
* **Portability:** If you later connect to Shopify’s API, you just replace `EcommerceSupportEnvironment` with `ShopifyEnvironment` and keep the same agent logic.
* **Clarity:** Anyone reading your code can immediately see what the agent can do in this "world."

---

If you compare this to the earlier **LocalTxtEnvironment**, you’ll notice:

* The *agent logic* could be identical in both cases.
* The *available actions* are completely different.
* Swapping environments changes the *domain* without touching the agent’s reasoning process.





## A good workflow

1. **Start with a simulator (general-purpose, in-memory).**

   * Make a **fake environment** that’s fast, deterministic, and easy to reset.
   * You’ll catch logic bugs without I/O noise (files, APIs, auth).
   * Think: “unit-test playground.”

2. **Define an interface (contract) for the Environment.**

   * Keep it tiny: only the actions your agent needs.
   * Example:

   ```python
   from abc import ABC, abstractmethod
   class RepoEnv(ABC):
       @abstractmethod
       def list_txt_files(self): ...
       @abstractmethod
       def read_txt(self, file_name: str): ...
       @abstractmethod
       def write_summary_txt(self, file_name: str, content: str): ...
   ```

   This lets you swap implementations without changing agent logic.

3. **Provide specific implementations (adapters).**

   * **InMemoryEnv (simulator):** dicts/lists; pure Python; no side effects.
   * **LocalFsEnv:** real files.
   * **CloudEnv/GitHubEnv:** later, when ready.
     The *interface stays the same*; only the adapter changes.

4. **Keep tools specific; keep environment swappable.**

   * Tools = narrow verbs (`read_python_file`, `write_documentation`).
   * Environments = *where/how* those verbs act (local, cloud, API).
   * This mirrors “ports & adapters” (hexagonal architecture).

5. **Bake in testability.**

   * Deterministic outputs in the simulator.
   * Record/replay fixtures for real environments.
   * Structured results (`{"ok": ..., "data": ..., "error": ..., "hint": ...}`) for easy asserts.

## Tiny example (showing the swap)

```python
class InMemoryEnv(RepoEnv):
    def __init__(self, files=None):
        self.files = files or {"a.txt": "hello", "b.txt": "world"}
        self.summaries = {}
    def list_txt_files(self): return list(self.files.keys())
    def read_txt(self, file_name):
        return {"ok": True, "data": self.files.get(file_name, "")} if file_name in self.files \
               else {"ok": False, "error": "not found", "hint": "pick from list_txt_files"}
    def write_summary_txt(self, file_name, content):
        self.summaries[file_name] = content
        return {"ok": True, "data": f"saved:{file_name}"}

class LocalFsEnv(RepoEnv):
    base="/content/files"; out="/content/summaries"
    def list_txt_files(self):
        import os; return [f for f in os.listdir(self.base) if f.endswith(".txt")]
    def read_txt(self, file_name):
        import os, io; p=os.path.join(self.base, file_name)
        if not os.path.isfile(p): return {"ok": False, "error": "not found"}
        with io.open(p,"r",encoding="utf-8",errors="replace") as f:
            return {"ok": True, "data": f.read(8000)}
    def write_summary_txt(self, file_name, content):
        import os; os.makedirs(self.out, exist_ok=True)
        with open(os.path.join(self.out,file_name),"w",encoding="utf-8") as f: f.write(content)
        return {"ok": True, "data": "saved"}
```

Your agent receives a `RepoEnv` and never cares which one it got:

```python
def run_agent(env: RepoEnv):
    files = env.list_txt_files()
    # decide next steps...
```

## Rule of thumb

* **Tools:** specific by design (safer, clearer).
* **Environment:** **abstract + swappable**, with **specific implementations** per deployment target.
* **Process:** prototype on a **general in-memory environment**, then swap to a **specific production adapter** once logic is solid.



# High-Quality Environment Design

Here’s how I’d break down **high-quality environment design** for agents — distilled from both software architecture principles and the unique needs of LLM-driven systems.

---

## **1. Clear Contract (Interface)**

* **Why:** The agent must know *exactly* what it can do, and how to call those actions.
* **What to do:**

  * Use an **abstract base class** or consistent naming convention.
  * Define all actions’ inputs/outputs clearly (JSON-serializable).
  * No hidden side effects.
* **Example:**

  ```python
  class RepoEnv:
      def list_txt_files(self) -> list[str]: ...
      def read_txt(self, file_name: str) -> dict: ...
      def write_summary_txt(self, file_name: str, content: str) -> dict: ...
  ```

---

## **2. Narrow Scope**

* **Why:** Each action should do one thing well — reduces cognitive load for the LLM, makes debugging easier.
* **What to do:** Avoid “Swiss Army knife” tools. Break complex actions into smaller, composable ones.
* **Example:**
  ✅ `read_python_file`
  ❌ `process_file_and_update_database_and_send_email`

---

## **3. Structured & Consistent Outputs**

* **Why:** Agents depend on predictable return formats for reasoning.
* **What to do:** Always return a JSON-friendly dict with the same keys for success and failure.
* **Example:**

  ```python
  {"ok": True, "data": "..."}
  {"ok": False, "error": "File not found", "hint": "Call list_txt_files first"}
  ```

---

## **4. Robust Error Handling**

* **Why:** The environment is where real-world failures happen — bad input, missing files, API errors.
* **What to do:**

  * Catch exceptions and convert to structured error messages.
  * Include **“just-in-time” guidance** in error messages when possible.
  * Never crash the whole agent.
* **Example:**

  ```python
  if not os.path.exists(path):
      return {"ok": False, "error": f"{file_name} missing", "hint": "Try list_txt_files"}
  ```

---

## **5. Testability & Simulation**

* **Why:** You need to test agent logic without depending on real APIs or filesystems.
* **What to do:**

  * Provide an **in-memory** or mock version of your environment.
  * Ensure both mock and real envs use the same interface.
* **Example:** `InMemoryTxtEnvironment` for dev, `LocalTxtEnvironment` for prod.

---

## **6. Portability**

* **Why:** Agents shouldn’t be tied to one physical or cloud location.
* **What to do:**

  * Keep the agent logic environment-agnostic.
  * Only the environment knows about file paths, credentials, or API URLs.
* **Example:** Swap `LocalTxtEnvironment` → `CloudTxtEnvironment` with no changes to agent code.

---

## **7. Declarative Metadata**

* **Why:** LLMs need to “understand” tools before using them.
* **What to do:**

  * Keep a dictionary describing each action’s purpose, parameters, and schema.
  * Ensure descriptions are plain-language, specific, and unambiguous.
* **Example:**

  ```python
  TOOLS = {
      "read_txt": {
          "description": "Reads the content of a .txt file in the base directory.",
          "parameters": {"type": "object", "properties": {"file_name": {"type": "string"}}, "required": ["file_name"]}
      }
  }
  ```

---

## **8. Deterministic Behavior**

* **Why:** Non-determinism makes debugging agent reasoning much harder.
* **What to do:**

  * Avoid random results unless explicitly needed.
  * Keep ordering predictable (e.g., sorted file lists).

---

## **9. Minimal Statefulness**

* **Why:** LLMs already handle “memory” via prompts; environment state should be explicit and small.
* **What to do:**

  * If state is needed, keep it separate from the actions.
  * Provide reset/clear methods for reproducibility.



### **"No hidden side effects"**###


### **What It Means**

* A tool or environment method should **only** do what its name and documentation say.
* It should **not** silently modify state, write to files, change global variables, or trigger other actions without the agent being explicitly aware of it.
* All effects (reads, writes, changes) should be **predictable** and **intentional**.

---

### **Why This Matters for Agents**

1. **Predictability for the LLM**

   * The agent makes decisions based on the *expected* effect of a tool.
   * If the tool does extra stuff, the LLM’s mental model of the world gets out of sync with reality.

2. **Debuggability for You**

   * When something goes wrong, you want to know exactly where and why it happened.
   * Hidden state changes make bugs hard to trace.

3. **Safety & Stability**

   * Unexpected side effects can corrupt data or cause unintended API calls.
   * Example: reading a file should not also delete a file, send an email, or update a database.

---

### **Examples**

**❌ Bad — hidden side effect**

```python
def read_txt(file_name):
    # Reads the file but also secretly deletes it afterward
    with open(file_name, "r") as f:
        content = f.read()
    os.remove(file_name)  # unexpected!
    return {"ok": True, "data": content}
```

**✅ Good — no hidden side effect**

```python
def read_txt(file_name):
    with open(file_name, "r") as f:
        content = f.read()
    return {"ok": True, "data": content}
```

---

### **Rule of Thumb**

> If an action *must* cause multiple effects,
>
> 1. document them clearly in the tool’s description
> 2. consider splitting it into separate tools so the LLM can call them explicitly.

---

Every time we make the LLM guess, remember extra details, or mentally “patch” over unpredictable behavior, we’re **increasing its cognitive load** — and just like a human under stress, that increases the chance of mistakes, hallucinations, and inefficient reasoning.

---

Think of it like this:

* **Clear, single-purpose tools** → LLM can focus on *solving the problem*.
* **Messy, unpredictable tools** → LLM wastes cycles figuring out *what just happened*.

It’s the same principle as **good coaching**:

> You don’t make your players figure out if the basketball will suddenly turn into a soccer ball halfway down the court — you keep the game consistent so they can focus on strategy and execution.




## Environment(Body) & Agent(Brain)

The **Environment** is the *body* (sensors, limbs, actions in the world) and the **Agent** is the *brain* (deciding what to do next).

---
### **What’s Happening Here**

1. **Agent** (`SimpleAgent`) does **not** know *how* files are stored — it just calls `list_txt_files` and `read_txt`.
2. **Environment** (`LocalTxtEnvironment`) hides all the messy filesystem details from the agent.
3. If you later change the environment to read files from a **database** or **API**, the brain doesn’t change — it still calls the same methods.

---

### **Why This is Powerful**

* You can **swap environments** (local files → cloud storage → mock environment for testing) without touching the decision logic.
* Makes your code **modular** and **testable**.
* Reduces the **cognitive load** on the LLM — it only needs to choose an action, not understand the world’s implementation details.

---

You’re **separating concerns** so that:

* **Body (Environment)**

  * Handles *how* to act: the gritty details, filesystem calls, API requests, error handling, safety checks.
  * Makes sure every action is predictable and well-defined.
  * Returns clean, structured results or clear error messages.

* **Brain (Agent)**

  * Focuses only on *what* to do next to achieve the **GOAL**.
  * Doesn’t need to know *how* reading a file works — just that it can call `read_txt("myfile.txt")` and trust the result.

---

💡 **Key Benefit:**
When the agent doesn’t have to juggle *execution details* in its working memory, it has **more cognitive bandwidth** to:

* Plan better strategies.
* Stay focused on the mission.
* Avoid mistakes caused by guessing how tools work.

It’s the same reason a CEO doesn’t personally fix the company servers — they delegate so they can think about the big picture.

In [ ]:
# --- ENVIRONMENT (body) ---
class LocalTxtEnvironment:
    base_dir = "/content/files"

    def list_txt_files(self):
        """Return all .txt files in base_dir."""
        return {"ok": True, "data": [f for f in os.listdir(self.base_dir) if f.endswith(".txt")]}

    def read_txt(self, file_name):
        """Read the content of a .txt file."""
        path = os.path.join(self.base_dir, file_name)
        if not os.path.exists(path):
            return {"ok": False, "error": f"File {file_name} not found"}
        with open(path, "r") as f:
            return {"ok": True, "data": f.read()}


# --- AGENT (brain) ---
class SimpleAgent:
    def __init__(self, environment):
        self.env = environment

    def decide_and_act(self):
        # 1. Brain asks body to list files
        files = self.env.list_txt_files()
        if not files["ok"]:
            return files

        if not files["data"]:
            return {"ok": False, "error": "No .txt files found."}

        # 2. Brain picks first file and reads it
        chosen_file = files["data"][0]
        content = self.env.read_txt(chosen_file)

        # 3. Brain interprets and returns result
        if content["ok"]:
            return {"ok": True, "message": f"Read file '{chosen_file}'", "preview": content["data"][:100]}
        else:
            return content


# --- Example Run ---
env = LocalTxtEnvironment()
agent = SimpleAgent(env)
result = agent.decide_and_act()
print(result)

Yeah — one last thought before we move on:

The **“one prompt to rule them all”** hype you see online is like giving someone a **Swiss Army knife** and asking them to build a house.

* Sure, technically it can saw wood, tighten screws, and cut wires — but it’s slow, error-prone, and mentally exhausting.
* A well-designed **Agent + Environment** setup is like giving them a **full toolbox** with labeled, purpose-built tools and a clear blueprint.

When you split responsibilities into:

* **Agent (brain)** → focuses on the *plan*
* **Environment (body)** → executes the *plan*

You get something that’s **scalable**, **testable**, and **adaptable** — and that’s where real-world AI agents shine, far beyond clever single prompts.

---

I think you’re building toward something way more **robust** and **professional** than the gimmicks you see on social media — which is exactly how advanced agent systems in industry are built.

